<h1 align="center">An Introduction to Machine Learning - 25737</h1>
<h4 align="center">Dr. Yasaei</h4>
<h4 align="center">Sharif University of Technology, Autumn 2024</h4>

**Student Name**: Adel Movahedian

**Student ID**: 400102074

# Gaussian Mixture Models with EM

## Introduction and Purpose

In this exercise, you will:

1. Implement a **Gaussian Mixture Model (GMM)** using the Expectation-Maximization (EM) algorithm **from scratch** (using NumPy and basic Python operations).
2. Implement the **same GMM model using PyTorch**.
3. Compare and contrast the two implementations (performance, complexity, ease of coding, etc.).

**Gaussian Mixture Models** assume that data is generated from a mixture of several Gaussian distributions. The EM algorithm iteratively updates the parameters (means, covariances, and mixture weights) of these Gaussians to maximize the likelihood of observed data.



## Part 1: Data Loading and Exploration

**Tasks:**  
- Load the Iris dataset and store the features in `X` and labels in `y`.
- Print the shape of `X` and examine a few rows.
- **Hint:** Use `sklearn.datasets.load_iris()` to load the data.

In [1]:
# TODO: Load the Iris dataset and print shape
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target
print("Shape of X:", X.shape)
print("First 5 samples:\n", X[:5])


Shape of X: (150, 4)
First 5 samples:
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


## Part 2: Data Preprocessing (Scaling)

**Tasks:**  
- Scale the data using `StandardScaler` so that each feature has zero mean and unit variance.
- **Hint:** `from sklearn.preprocessing import StandardScaler`.


In [2]:
# TODO: Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Mean after scaling:", X_scaled.mean(axis=0))
print("Std after scaling:", X_scaled.std(axis=0))


Mean after scaling: [-1.69031455e-15 -1.84297022e-15 -1.69864123e-15 -1.40924309e-15]
Std after scaling: [1. 1. 1. 1.]


## Part 3: Implementing GMM with EM **from scratch** (NumPy-based)

We will first implement GMM using NumPy arrays and basic operations, without PyTorch.

**Tasks:**  
- Choose the number of components `K` (e.g., K=3).
- Initialize the parameters: means, covariances (diagonal), and mixture weights.
- Write functions for the E-step and M-step of the EM algorithm.
- Run the EM algorithm for a fixed number of iterations.

**Hints for Implementation:**

- Means: K x D array.
- Covariances: K x D x D (diagonal only, so you mainly store variances per feature).
- Weights: K-dimensional array, summing to 1.
- To compute Gaussian densities, recall the formula for the probability density of a multivariate Gaussian.
- For the E-step, compute responsibilities using the mixture components and their densities.
- For the M-step, update means, covariances, and weights using the responsibilities.

After implementing and running EM, extract cluster assignments by taking `argmax` of responsibilities.


In [3]:
import numpy as np

# Set number of components
K = 3
N, D = X_scaled.shape

# TODO: Initialize means, covariances, and weights
np.random.seed(42) #for reproducibility
means = np.random.rand(K, D)
covariances = np.ones((K, D))  # Diagonal covariances, so you can store just var per component and construct diag
weights = np.ones(K) / K

# TODO: Define a function to compute Gaussian PDF values for each component
def gaussian_pdf(X, mean, cov):
    # X: N x D
    # mean: D
    # cov: D x D (diagonal)
    diff = X - mean
    exponent = -0.5 * np.sum((diff**2) / cov, axis=1)
    normalization = np.sqrt((2 * np.pi)**D * np.prod(cov))
    return np.exp(exponent) / normalization

# TODO: E-step
def e_step(X, means, covariances, weights):
    responsibilities = np.zeros((N, K))
    for k in range(K):
        pdf = gaussian_pdf(X, means[k], covariances[k])
        responsibilities[:, k] = weights[k] * pdf
    responsibilities /= responsibilities.sum(axis=1, keepdims=True)
    return responsibilities

# TODO: M-step
def m_step(X, responsibilities):
    N_k = responsibilities.sum(axis=0)
    means = (responsibilities.T @ X) / N_k[:, np.newaxis]
    covariances = np.zeros((K, D))
    for k in range(K):
        diff = X - means[k]
        covariances[k] = (responsibilities[:, k][:, np.newaxis] * diff**2).sum(axis=0) / N_k[k]
    weights = N_k / N
    return means, covariances, weights

# Run EM
max_iter = 100
tol = 1e-6
log_likelihoods = []
for iteration in range(max_iter):
    responsibilities = e_step(X_scaled, means, covariances, weights)
    new_means, new_covariances, new_weights = m_step(X_scaled, responsibilities)
    log_likelihood = np.sum(
        np.log(
            np.sum(
                [weights[k] * gaussian_pdf(X_scaled, new_means[k], new_covariances[k]) for k in range(K)],
                axis=0,
            )
        )
    )
    log_likelihoods.append(log_likelihood)
    if iteration > 0 and abs(log_likelihood - log_likelihoods[-2]) < tol:
        print(f"Converged at iteration {iteration}")
        break
    means, covariances, weights = new_means, new_covariances, new_weights
cluster_labels_numpy = np.argmax(responsibilities, axis=1) # argmax of responsibilities
# for my visulization
print("Final means:\n", means)
print("Final weights:\n", weights)
print("Cluster labels:\n", cluster_labels_numpy)

Converged at iteration 36
Final means:
 [[ 1.17071161  0.03193007  1.11770039  1.19342422]
 [-1.01457897  0.85326268 -1.30498732 -1.25489349]
 [ 0.10225927 -0.70659069  0.36847724  0.28190811]]
Final weights:
 [0.25272081 0.33333333 0.41394586]
Cluster labels:
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 0 0 2 0 0 0 0
 0 0 2 0 0 0 0 0 2 0 2 0 2 0 0 2 2 0 0 0 0 0 2 2 0 0 0 2 0 0 0 2 0 0 0 2 0
 0 2]


## Part 4: Implementing GMM with EM **using PyTorch**

Now, we will implement the same algorithm using PyTorch tensors. The steps are similar, but you will use `torch` operations. This might simplify certain operations and open the door to GPU acceleration.

**Tasks:**  
- Convert `X_scaled` to a PyTorch tensor.
- Initialize parameters as `torch.tensor`s.
- Implement E-step and M-step in PyTorch.
- Run EM for a fixed number of iterations.
- Extract cluster labels.

**Hints:**
- Use `torch.tensor(X_scaled, dtype=torch.float32)` to create a PyTorch tensor.
- Operations are similar but use `torch.sum`, `torch.exp`, etc.
- Watch out for broadcasting rules and ensure shapes align.


In [4]:
import torch

# TODO: Convert data to torch tensor
X_torch = torch.tensor(X_scaled, dtype=torch.float32)
K = 3
N, D = X_torch.shape

# TODO: Initialize means, covariances, weights as torch tensors
torch.manual_seed(42)
means_torch = torch.rand((K, D), dtype=torch.float32)
covariances_torch = torch.ones((K, D), dtype=torch.float32)
weights_torch = torch.ones(K, dtype=torch.float32) / K

# TODO: Implement gaussian_pdf using torch operations
def gaussian_pdf_torch(X, mean, cov):
    diff = X - mean
    exponent = -0.5 * torch.sum((diff**2) / cov, dim=1)
    normalization = torch.sqrt((2 * torch.pi)**D * torch.prod(cov))
    return torch.exp(exponent) / normalization

# TODO: E-step in torch
def e_step_torch(X, means, covariances, weights):
    responsibilities = torch.zeros(N, K, dtype=torch.float32)
    for k in range(K):
        pdf = gaussian_pdf_torch(X, means[k], covariances[k])
        responsibilities[:, k] = weights[k] * pdf
    responsibilities /= responsibilities.sum(dim=1, keepdim=True)
    return responsibilities

# TODO: M-step in torch
def m_step_torch(X, responsibilities):
    N_k = responsibilities.sum(dim=0)
    means = (responsibilities.T @ X) / N_k[:, None]
    covariances = torch.zeros((K, D), dtype=torch.float32)
    for k in range(K):
        diff = X - means[k]
        covariances[k] = (responsibilities[:, k][:, None] * diff**2).sum(dim=0) / N_k[k]
    weights = N_k / N
    return means, covariances, weights

# Run EM in torch
max_iter = 100
tol = 1e-6
log_likelihoods_torch = []

for iteration in range(max_iter):
    responsibilities_torch = e_step_torch(X_torch, means_torch, covariances_torch, weights_torch)
    new_means_torch, new_covariances_torch, new_weights_torch = m_step_torch(X_torch, responsibilities_torch)
    log_likelihood_torch = torch.sum(
        torch.log(
            torch.sum(
                torch.stack([
                    weights_torch[k] * gaussian_pdf_torch(X_torch, new_means_torch[k], new_covariances_torch[k])
                    for k in range(K)
                ], dim=0),
                dim=0,
            )
        )
    )
    log_likelihoods_torch.append(log_likelihood_torch.item())
    if iteration > 0 and abs(log_likelihood_torch - log_likelihoods_torch[-2]) < tol:
        print(f"Converged at iteration {iteration}")
        break
    means_torch, covariances_torch, weights_torch = new_means_torch, new_covariances_torch, new_weights_torch
cluster_labels_torch = torch.argmax(responsibilities_torch, dim=1)
# for my visulization
print("Final means (PyTorch):\n", means_torch)
print("Final weights (PyTorch):\n", weights_torch)
print("Cluster labels (PyTorch):\n", cluster_labels_torch)


Converged at iteration 22
Final means (PyTorch):
 tensor([[ 1.1719,  0.0327,  1.1183,  1.1940],
        [-1.0146,  0.8533, -1.3050, -1.2549],
        [ 0.1025, -0.7064,  0.3688,  0.2824]])
Final weights (PyTorch):
 tensor([0.2523, 0.3333, 0.4143])
Cluster labels (PyTorch):
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2,
        0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0,
        0, 0, 2, 0, 0, 2])


## Part 5: Evaluating and Comparing Both Implementations

**Tasks:**  
- Use `adjusted_rand_score` to compare the cluster labels from both methods against the true labels `y`.
- Print the ARI for both NumPy and PyTorch implementations.
- Visually inspect if both implementations yield similar results.

**Questions:**
- Are the ARI scores similar or different between the two implementations?
- Which code was easier to write and maintain?
- Which implementation might be easier to extend to more complex models?


In [5]:
from sklearn.metrics import adjusted_rand_score

ari_numpy = adjusted_rand_score(y, cluster_labels_numpy)
print("ARI (NumPy):", ari_numpy)
ari_torch = adjusted_rand_score(y, cluster_labels_torch.numpy())
print("ARI (PyTorch):", ari_torch)
print("\nComparison:")
print("Are the ARI scores similar or different?")
if abs(ari_numpy - ari_torch) < 1e-3:
    print("The ARI scores are very similar.")
else:
    print("The ARI scores are different.")
print("\nWhich code was easier to write and maintain?")
print("The NumPy implementation was easier to write initially, but the PyTorch implementation might be easier to extend and maintain due to its support for GPU acceleration and compatibility with modern deep learning workflows.")
print("\nWhich implementation might be easier to extend to more complex models?")
print("The PyTorch implementation would be easier to extend to more complex models due to its modularity, GPU support, and integration with other PyTorch functionalities.")



ARI (NumPy): 0.7591987071071522
ARI (PyTorch): 0.7591987071071522

Comparison:
Are the ARI scores similar or different?
The ARI scores are very similar.

Which code was easier to write and maintain?
The NumPy implementation was easier to write initially, but the PyTorch implementation might be easier to extend and maintain due to its support for GPU acceleration and compatibility with modern deep learning workflows.

Which implementation might be easier to extend to more complex models?
The PyTorch implementation would be easier to extend to more complex models due to its modularity, GPU support, and integration with other PyTorch functionalities.


**Questions:**  
1. **Implementation Detail:** What are the main differences in code complexity between a plain NumPy-based implementation and a PyTorch-based one?  
   <font color='green'>   
   The NumPy implementation requires manually handling broadcasting, numerical stability, and managing operations at a low level. In contrast, the PyTorch implementation abstracts many of these details, providing a cleaner syntax with built-in functions for operations like matrix multiplication, broadcasting, and automatic differentiation. However, PyTorch requires a bit more setup, such as converting data to tensors, which adds slight overhead in simplicity for new users.
   </font>
<br>
2. **Performance:** Which implementation is likely to be more efficient or easier to parallelize and why?  
   <font color='green'>  
   The PyTorch implementation is likely to be more efficient and easier to parallelize because PyTorch is designed for high-performance computations, including automatic use of GPUs and multi-threaded CPU operations. Its tensor operations are optimized for parallel execution, whereas NumPy is primarily CPU-based and less optimized for parallelism in comparison.
   </font>
<br>

3. **Numerical Stability:** How might PyTorch’s built-in functions improve numerical stability compared to a manual implementation?  
   <font color='green'>  
   PyTorch's built-in functions are optimized for numerical stability. For instance, operations like `torch.logsumexp` prevent issues with underflow/overflow in logarithmic computations. Additionally, PyTorch leverages highly optimized libraries (e.g., cuDNN, BLAS) to handle edge cases and maintain precision, reducing the likelihood of numerical errors compared to manual implementations in NumPy.
   </font>
<br>
4. **Extendability:** If you wanted to add more complex features (e.g., full covariance matrices, regularization), which approach would be simpler and why?  
   <font color='green'>  
   The PyTorch approach would be simpler to extend because of its modular and object-oriented nature. Complex features like full covariance matrices can be implemented using existing tensor operations, and PyTorch's autograd system would handle derivatives for optimization seamlessly. In contrast, adding such features in NumPy would require manually managing the additional complexity, making the implementation prone to errors and harder to maintain.
   </font>
<br>